In [ ]:
# default_exp core

# Swoggle

> TODO: API details. 

In [ ]:
#hide
from nbdev.showdoc import *

# The basic ideas

The board is an 8x8 grid of cells. We'll use a 'Cell' class to represent them, with the following attributes:

cell = {'x', 'y', 'drone':bool, 'player':PID, 'base':PID}

In [ ]:
# export
class Cell:
    """ Represent a cell on the game board """
    x, y = 0, 0
    drone = False
    player = None
    base = None
    
    def __init__(self, x, y, drone=False, player=None, base=None):
        self.x = x
        self.y = y
        self.drone = drone
        self.player = player
        self.base = base
        
    def is_occupied(self):
        if self.player != None:
            return True
        return False
    
    def is_free(self):
        if (self.player == None) and (self.base == None) and not (self.drone):
            return True
        return False

In [ ]:
# Test is_occupied
c  = Cell(1, 1)
c.player = 123
assert(c.is_occupied())
c.player = None
assert(c.is_occupied() == False) 

In [ ]:
# Test is_free
c  = Cell(1, 1)
assert(c.is_free()) 
c.player = 123
assert(c.is_free() == False)
c.player = None
c.drone = True
assert(c.is_free() == False) 
c.drone = None
c.base = 1
assert(c.is_free() == False) 

The above two run as tests - integrated in the NB, and easy to play with interactively.

### The board representation

A matrix of cells (the board), as well as the jail (for captured players).

Also need a way to do moves. Done with move(player, start, end, dice roll, drone, powerjump) where drone and powerjump are special moves.

In [ ]:
#export
import math

class Board:
    
    board = [[Cell(x, y) for x in range(8)] for y in range(8)]
    jail = []

    def is_valid_move(self, player, start_loc, end_loc, dice_roll, drone, powerjump):
        xs, ys, xe, ye = *start_loc, *end_loc

        # Check it's in bounds
        if max(xs, ys, xe, ye) > 7 or min(xs, ys, xe, ye) < 0:
            return False

        # Check that player is in start loc
        if self.board[xs][ys].player != player:
            print(2)
            return False

        # Check that there's a drone in the start loc if drone
        if drone and not self.board[xs][ys].drone:
            return False
        
        #Check they're starting from their base if powerjump
        if powerjump and not self.board[xs][ys].base == player:
            return False

        # Adjust max distance if drone or powerjump
        if drone or powerjump:
            dice_roll = math.ceil(dice_roll/2)

        # Check we're in range
        if abs(xe-xs) > dice_roll:
            return False
        if abs(ye-ys) > dice_roll:
            return False

        # Check the base isn't defended against an un-armoured attacker
        if self.board[xe][ye].base != None:
            if self.board[xe][ye].player and drone==False: # Defended
                return False

        # Check if dest is droned player
        if (self.board[xe][ye].player != None) and (self.board[xe][ye].drone) and not (drone or powerjump):
            return False
        
        # If you rolled a 1, you have to move 1
        if dice_roll == 1:
            if (xe==xe)


        # Maaaaaybe see if there are drones in the way?

        return True

    def move(self, player, start_loc, end_loc, dice_roll, drone=False, powerjump=False):
        
        # If in jail, try to escape (5 or 6)
        if player in self.jail:
            if dice_roll > 4:
                self.jail.remove(player)
                # Put player back on base:
                for row in self.board:
                    for cell in row:
                        if cell.base == player:
                            cell.player = player
                print('Player', player, 'escaped')
                # End turn
                return True
            else:
                print('Player', player, 'did not escape')
                return True
        
        # Check move validity
        if not self.is_valid_move(player, start_loc, end_loc, dice_roll, drone, powerjump):
            print('Invalid Move', player, start_loc, end_loc)
            return False

        xs, ys, xe, ye = *start_loc, *end_loc
        
        # Check for player clashes
        if self.board[xe][ye].player != None: # Valid move so we capture them
            pid = self.board[xe][ye].player
            print('Player', pid, 'sent to Swoggle Spa')
            self.board[xe][ye].player = None
            self.jail.append(pid)

        # Move player
        self.board[xs][ys].player = None
        self.board[xe][ye].player = player
        
        # Optional - should powerjumping to a droned player destroy the drone? Comment for no
        if powerjump:
            if self.board[xe][ye].drone:
                print('Drone destroyed')
            self.board[xe][ye].drone = False

        # Move drone
        if drone:
            if not self.board[xe][ye].drone: # If there's a drone there already, leave ours behind 
                self.board[xs][ys].drone = False
            self.board[xe][ye].drone = True

        # Check for base defeats
        if self.board[xe][ye].base != None: # If move is valid, we defeated the base
            # Player goes to own base, base and player of defeated charachter disappear
            pid = self.board[xe][ye].base
            print("Player", str(player), "defeated player", pid)
            self.board[xe][ye].base = None
            self.board[xe][ye].player = None
            for row in self.board:
                for cell in row:
                    # Clear defeated charachetr
                    if cell.player == pid:
                        cell.player = None
                    # Return attacking player
                    if cell.base == player:
                        cell.player = player


        print('Moved', player, start_loc, end_loc)
        return True

    def show(self):
        for row in self.board:
            s = ''
            for cell in row:
                s += '['
                if cell.player != None:
                    s += str(cell.player)
                else:
                    s += '.'
                if cell.drone:
                    s += 'd'
                else:
                    s += '.'
                if cell.base != None:
                    s += str(cell.base)
                else:
                    s += '.'
                s += ']'
            print(s)

### Trial game

In [ ]:
# Create a board
b = Board()

# Add some bases and draw starting setup
b.board[0][0].player = 1
b.board[0][0].base = 1 
b.board[7][7].player = 2
b.board[7][7].base = 2
b.board[3][3].drone = True
b.show()

print('next')

b.move(1, (0, 0), (1, 1), 1)
b.show()
b.move(1, (1, 1), (6, 6), 4)
b.show()
b.move(1, (1, 1), (4, 4), 5, drone=True) # Fails  - no drone
b.show()
b.move(1, (1, 1), (3, 3), 5, drone=False) # Start from drone loc
b.move(1, (3, 3), (4, 4), 5, drone=True) # Works
b.show()

[1.1][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][2.2]
next
Moved 1 (0, 0) (1, 1)
[..1][...][...][...][...][...][...][...]
[...][1..][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][2.2]
Invalid Move 1 (1, 1) (6, 6)
[..1][...][...][...][...][...][...][...]
[...][1..][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][.d.][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[

In [ ]:
# b.move(1, (4, 4), (7, 7), 5, drone=True) # P1 defeats P2
b.move(2, (7, 7), (4, 4), 5, powerjump=True) # Testing powerjump
b.show()

Player 1 sent to Swoggle Spa
Drone destroyed
Moved 2 (7, 7) (4, 4)
[..1][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][2..][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][..2]


In [ ]:
b.move(1, (0, 0), (0, 0), 5) # Doesn't escape with 4 or below
b.show()

Player 1 escaped
[1.1][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][2..][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][..2]


In [ ]:
b.move(1, (0, 0), (4, 4), 5)
b.show()

Player 2 sent to Swoggle Spa
Moved 1 (0, 0) (4, 4)
[..1][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][1..][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][..2]


In [ ]:
b.move(1, (4, 4), (7, 7), 5)
b.show()

Player 1 defeated player 2
Moved 1 (4, 4) (7, 7)
[1.1][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]
[...][...][...][...][...][...][...][...]


TODO:
Have to move 1 square if you roll a 1
Can move ont own base without defeating oneself!

In [ ]:
# Start at 15:22